Leitura dos Dados
Primeiro, vamos carregar o dataset de vinhos da UCI:

In [ ]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Verificando os dados
print(vinhos.head())


  classe  alcool  acido_malico  cinzas  alcalinidade_de_cinzas  magnesio  \
0      1   14.23          1.71    2.43                    15.6       127   
1      1   13.20          1.78    2.14                    11.2       100   
2      1   13.16          2.36    2.67                    18.6       101   
3      1   14.37          1.95    2.50                    16.8       113   
4      1   13.24          2.59    2.87                    21.0       118   

   fenois_totais  flavanoides  fenois_nao_flavanoides  proantocianinas  \
0           2.80         3.06                    0.28             2.29   
1           2.65         2.76                    0.26             1.28   
2           2.80         3.24                    0.30             2.81   
3           3.85         3.49                    0.24             2.18   
4           2.80         2.69                    0.39             1.82   

   intensidade_de_cor  matiz  od280_od315_de_vinhos_diluidos  prolina  
0                5.64   1.

Pré-processamento dos Dados
Separemos as variáveis de entrada (X) e saída (y):


In [ ]:
# Separar os dados em variáveis de entrada (X) e saída (y)
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']


Configuração do Experimento
Realizaremos três experimentos, alterando o random_state para os valores 42, 17 e 24. Para cada experimento, aplicaremos o KNN com k=3 e k=5.

4. Estrutura de Cross-Validation com KFold
Utilizaremos o KFold para realizar a validação cruzada. Vamos estruturar o código para aplicar o KNN e calcular a acurácia em cada fold:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def avaliar_knn(X, y, random_state):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    accuracies_k3 = []
    accuracies_k5 = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # KNN com k=3
        knn3 = KNeighborsClassifier(n_neighbors=3)
        knn3.fit(X_train, y_train)
        y_pred_3 = knn3.predict(X_test)
        accuracies_k3.append(accuracy_score(y_test, y_pred_3))

        # KNN com k=5
        knn5 = KNeighborsClassifier(n_neighbors=5)
        knn5.fit(X_train, y_train)
        y_pred_5 = knn5.predict(X_test)
        accuracies_k5.append(accuracy_score(y_test, y_pred_5))

    return {
        'random_state': random_state,
        'mean_accuracy_k3': np.mean(accuracies_k3),
        'std_accuracy_k3': np.std(accuracies_k3),
        'mean_accuracy_k5': np.mean(accuracies_k5),
        'std_accuracy_k5': np.std(accuracies_k5),
    }

# Avaliar com random_state 42, 17 e 24
resultados_42 = avaliar_knn(X, y, 42)
resultados_17 = avaliar_knn(X, y, 17)
resultados_24 = avaliar_knn(X, y, 24)

print(resultados_42)
print(resultados_17)
print(resultados_24)


{'random_state': 42, 'mean_accuracy_k3': 0.70359477124183, 'std_accuracy_k3': 0.13918691592617136, 'mean_accuracy_k5': 0.6640522875816993, 'std_accuracy_k5': 0.10007472964927165}
{'random_state': 17, 'mean_accuracy_k3': 0.6986928104575163, 'std_accuracy_k3': 0.10403044405660643, 'mean_accuracy_k5': 0.6875816993464052, 'std_accuracy_k5': 0.11211770116307625}
{'random_state': 24, 'mean_accuracy_k3': 0.7127450980392157, 'std_accuracy_k3': 0.09681644876344142, 'mean_accuracy_k5': 0.7127450980392157, 'std_accuracy_k5': 0.09397285976504524}


Análise dos Resultados
Após a execução dos experimentos, vamos calcular a média e o desvio padrão da acurácia para cada valor de k em cada experimento e identificar a melhor configuração:

In [ ]:
resultados = [resultados_42, resultados_17, resultados_24]

for resultado in resultados:
    print(f"random_state={resultado['random_state']} -> "
          f"Média Acurácia k=3: {resultado['mean_accuracy_k3']:.4f} (+/- {resultado['std_accuracy_k3']:.4f}), "
          f"Média Acurácia k=5: {resultado['mean_accuracy_k5']:.4f} (+/- {resultado['std_accuracy_k5']:.4f})")

melhor_configuracao = max(resultados, key=lambda x: max(x['mean_accuracy_k3'], x['mean_accuracy_k5']))
print(f"Melhor configuração: random_state={melhor_configuracao['random_state']}, "
      f"k={'3' if melhor_configuracao['mean_accuracy_k3'] > melhor_configuracao['mean_accuracy_k5'] else '5'}, "
      f"Acurácia={max(melhor_configuracao['mean_accuracy_k3'], melhor_configuracao['mean_accuracy_k5']):.4f}")


random_state=42 -> Média Acurácia k=3: 0.7036 (+/- 0.1392), Média Acurácia k=5: 0.6641 (+/- 0.1001)
random_state=17 -> Média Acurácia k=3: 0.6987 (+/- 0.1040), Média Acurácia k=5: 0.6876 (+/- 0.1121)
random_state=24 -> Média Acurácia k=3: 0.7127 (+/- 0.0968), Média Acurácia k=5: 0.7127 (+/- 0.0940)
Melhor configuração: random_state=24, k=5, Acurácia=0.7127
